## Requirements
- Identifies which of the three proposals you outlined in your lightning talk you have chosen
- Articulates the main goal of your project (your problem statement)
- Outlines your proposed methods and models
- Defines the risks & assumptions of your data
- Revises initial goals & success criteria, as needed
- Documents your data source


## Problem Statement
- Specific:
  - What precisely do you plan to do?
  - What type of model will you need to develop?
- Measurable:
  - What metrics will you be using to assess performance?
  - MSE? Accuracy? Precision? AUC?
- Achievable:
  - Is your project appropriately scoped?
  - Is it too aggressive?  Too easy?
  - *Note:* If your project is too big, break it up into smaller pieces.  Sometimes a good project is the simply one part of a larger, longer-term agenda.
- Relevant:
  - Does anyone care about this?
  - Why should people be interested in your results?
  - What value will the completion of your project be adding?
- Time-bound
  - What's your deadline?

### Main goal & Problem statement

My project will be building a stock picker that utilizes fundamental data from the companies' financial statements for selection. The main goal would be to pick a portfolio of ten stocks that is able to outperform the market benchmark (S&P500). Since quarterly data is published, we are more interested in the stock's performance after a year, and this will not be used as a trading strategy. This will also be a regression problem, as we are dealing with predicting stock performance.

A big bulk of the work will be spent on feature engineering of the fundamental data (e.g. financial and accounting ratios), so that these features will correlate with business success and be good performance indicators. Working with time series is also important as we are interested in the performance of the stock when compared to its financial performance.

### Proposed methods and models

I will work with models learnt throughout the course that show predictive ability on stock performance, and use them again for backtesting. We can use linear regression, regularized regression (Lasso or Ridge or ENet), KNN, SVM Regressor, Decision Trees as potential models. MSE will likely be the metric to assess the model performance.


Feature importance is also something to look at as we want to know which variables/ratios gives us better predictive ability for stock performance. After running these models, we can put them in a table to view the results of the top 10 selected stock portfolios.

After this, we will do backtesting to find out the volatility as well as the return profile of each portfolio. Backtesting will follow roughly these steps:
- split into train test sets and train model on training set
- for given year, use model to predict stock returns and pick 10 top performances from test set to create equal weighted portfolio
- record daily/weekly portfolio value change to see performance of portfolio
- repeat for the number of years of available data, likely using loops? (we should get return and volatility from here)

From all the models, we should get some kind of return/volatility profile, which we can then use to determine the better stocks to pick.

### Relevance

Potentially, this can be used by portfolio managers or retail investors who wish to do better than market returns without taking on too much additional risk. As I do not have any idea how the tested models would do at this point in time, I generally hope that results would at least meet the benchmark return at its minimum. The number of stocks picked is also kept small to minimize potential transaction costs.

### Timeline

I will start working on the project right after the completion of Project 4 and aim to come up with a working prototype by the 2nd week of capstone project start. I will try to refine the model in the third week and do up the presentation in the 4th week.

## Project starts here

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

%matplotlib inline

Simfin is where we will be downloading our fundamentals data from. Instead of downloading a CSV file, we will use the API key to download directly.

In [2]:
import simfin as sf

# Set your SimFin + API-key for downloading data.
sf.set_api_key('nYdjeL237mt7QaQ4OZTaPa4Xn3YECrTH')

# Set the local directory where data-files are stored.
# The directory will be created if it does not already exist.
sf.set_data_dir('~/simfin_data/')

# Download the data from the SimFin server and load into a Pandas DataFrame.
# We will be getting 3 sets of financial statements (income statement, balance sheet, cashflow statement)
df_income = sf.load_income(variant='quarterly', market='us')
df_balance = sf.load_balance(variant='quarterly', market='us')
df_cashflow = sf.load_cashflow(variant='quarterly', market='us')

# Print the first rows of the data.
df_income.head()

Dataset "us-income-quarterly" on disk (26 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (26 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (26 days old).
- Loading from disk ... Done!


SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2016-10-31      45846      USD         2016            Q4   2016-12-20   
       2017-01-31      45846      USD         2017            Q1   2017-03-08   
       2017-04-30      45846      USD         2017            Q2   2017-06-06   
       2017-07-31      45846      USD         2017            Q3   2017-09-06   
       2017-10-31      45846      USD         2017            Q4   2017-12-21   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2016-10-31     2018-12-20     324000000.0       328000000.0   
       2017-01-31     2018-03-06     322000000.0       326000000.0   
       2017-04-30     2018-05-31     321000000.0       325000000.0   
       2017-07-31     2018-08-30     321000000.0       326000000.0   
       2017-10-31     2018-12-20     324000000.0       327000000.0   

                         Revenue  Cost of Revenue  ...  \
Ticker Report Date                                 ...   
A      2016-10-31   1.111000e+09     -523000000.0  ...   
       2017-01-31   1.067000e+09     -493000000.0  ...   
       2017-04-30   1.102000e+09     -510000000.0  ...   
       2017-07-31   1.114000e+09     -518000000.0  ...   
       2017-10-31   1.189000e+09     -542000000.0  ...   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Report Date                                                       
A      2016-10-31                   -32000000.0            -16000000.0   
       2017-01-31                   -13000000.0            -16000000.0   
       2017-04-30                   -10000000.0            -15000000.0   
       2017-07-31                    -8000000.0            -13000000.0   
       2017-10-31                    -7000000.0            -13000000.0   

                    Pretax Income (Loss), Adj.  Abnormal Gains (Losses)  \
Ticker Report Date                                                        
A      2016-10-31                    151000000                      NaN   
       2017-01-31                    193000000                      NaN   
       2017-04-30                    191000000                      NaN   
       2017-07-31                    193000000                      NaN   
       2017-10-31                    226000000                      NaN   

                    Pretax Income (Loss)  Income Tax (Expense) Benefit, Net  \
Ticker Report Date                                                            
A      2016-10-31              151000000                        -25000000.0   
       2017-01-31              193000000                        -25000000.0   
       2017-04-30              191000000                        -27000000.0   
       2017-07-31              193000000                        -18000000.0   
       2017-10-31              226000000                        -49000000.0   

                    Income (Loss) from Continuing Operations  \
Ticker Report Date                                             
A      2016-10-31                                  126000000   
       2017-01-31                                  168000000   
       2017-04-30                                  164000000   
       2017-07-31                                  175000000   
       2017-10-31                                  177000000   

                    Net Extraordinary Gains (Losses)  Net Income  \
Ticker Report Date                                                 
A      2016-10-31                                NaN   126000000   
       2017-01-31                                NaN   168000000   
       2017-04-30                                NaN   164000000   
       2017-07-31                                NaN   175000000   
       2017-10-31                                NaN   177000000   

                    Net Incom

In [3]:
df_balance.head()

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2016-10-31      45846      USD         2016            Q4   2016-12-20   
       2017-01-31      45846      USD         2017            Q1   2017-03-08   
       2017-04-30      45846      USD         2017            Q2   2017-06-06   
       2017-07-31      45846      USD         2017            Q3   2017-09-06   
       2017-10-31      45846      USD         2017            Q4   2017-12-21   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2016-10-31     2017-12-21     324000000.0       328000000.0   
       2017-01-31     2017-03-08     322000000.0       326000000.0   
       2017-04-30     2017-06-06     321000000.0       325000000.0   
       2017-07-31     2017-09-06     321000000.0       326000000.0   
       2017-10-31     2018-12-20     324000000.0       327000000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Report Date                                                    
A      2016-10-31                                      2.289000e+09   
       2017-01-31                                      2.241000e+09   
       2017-04-30                                      2.389000e+09   
       2017-07-31                                      2.563000e+09   
       2017-10-31                                      2.678000e+09   

                    Accounts & Notes Receivable  ...  Short Term Debt  \
Ticker Report Date                               ...                    
A      2016-10-31                   631000000.0  ...              0.0   
       2017-01-31                   653000000.0  ...      190000000.0   
       2017-04-30                   677000000.0  ...      241000000.0   
       2017-07-31                   678000000.0  ...      280000000.0   
       2017-10-31                   724000000.0  ...      210000000.0   

                    Total Current Liabilities  Long Term Debt  \
Ticker Report Date                                              
A      2016-10-31                   945000000    1.904000e+09   
       2017-01-31                  1089000000    1.802000e+09   
       2017-04-30                  1187000000    1.802000e+09   
       2017-07-31                  1241000000    1.801000e+09   
       2017-10-31                  1263000000    1.801000e+09   

                    Total Noncurrent Liabilities  Total Liabilities  \
Ticker Report Date                                                    
A      2016-10-31                   2.603000e+09         3548000000   
       2017-01-31                   2.483000e+09         3572000000   
       2017-04-30                   2.454000e+09         3641000000   
       2017-07-31                   2.409000e+09         3650000000   
       2017-10-31                   2.328000e+09         3591000000   

                    Share Capital & Additional Paid-In Capital  \
Ticker Report Date                                               
A      2016-10-31                                 9.165000e+09   
       2017-01-31                                 5.239000e+09   
       2017-04-30                                 5.242000e+09   
       2017-07-31                                 5.285000e+09   
       2017-10-31                                 5.303000e+09   

                    Treasury Stock  Retained Earnings  Total Equity  \
Ticker Report Date                                                    
A      2016-10-31    -1.050800e+10       6.089000e+09  4.246000e+09   
       2017-01-31     0.000000e+00      -4.530000e+08  4.300000e+09   
       2017-04-30     0.000000e+00      -3.930000e+08  4.375000e+09   
       2017-07-31     0.000000e+00      -2.600000e+08  4.611000e+09   
       2017-10-31              NaN      -1.260000e+08  4.835000e+09   

                    Total Liabilitie

In [4]:
df_cashflow.head()

SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
Ticker Report Date                                                              
A      2016-10-31      45846      USD         2016            Q4   2016-12-20   
       2017-01-31      45846      USD         2017            Q1   2017-03-08   
       2017-04-30      45846      USD         2017            Q2   2017-06-06   
       2017-07-31      45846      USD         2017            Q3   2017-09-06   
       2017-10-31      45846      USD         2017            Q4   2017-12-21   

                   Restated Date  Shares (Basic)  Shares (Diluted)  \
Ticker Report Date                                                   
A      2016-10-31     2018-12-20     324000000.0       328000000.0   
       2017-01-31     2018-03-06     322000000.0       326000000.0   
       2017-04-30     2018-05-31     321000000.0       325000000.0   
       2017-07-31     2018-08-30     321000000.0       326000000.0   
       2017-10-31     2018-12-20     324000000.0       327000000.0   

                    Net Income/Starting Line  Depreciation & Amortization  \
Ticker Report Date                                                          
A      2016-10-31                126000000.0                   56000000.0   
       2017-01-31                168000000.0                   55000000.0   
       2017-04-30                164000000.0                   54000000.0   
       2017-07-31                175000000.0                   51000000.0   
       2017-10-31                177000000.0                   52000000.0   

                    ...  Net Cash from Operating Activities  \
Ticker Report Date  ...                                       
A      2016-10-31   ...                         234000000.0   
       2017-01-31   ...                         116000000.0   
       2017-04-30   ...                         257000000.0   
       2017-07-31   ...                         228000000.0   
       2017-10-31   ...                         288000000.0   

                    Change in Fixed Assets & Intangibles  \
Ticker Report Date                                         
A      2016-10-31                            -52000000.0   
       2017-01-31                            -32000000.0   
       2017-04-30                            -43000000.0   
       2017-07-31                            -43000000.0   
       2017-10-31                            -58000000.0   

                    Net Change in Long Term Investment  \
Ticker Report Date                                       
A      2016-10-31                                  0.0   
       2017-01-31                                  NaN   
       2017-04-30                                  NaN   
       2017-07-31                                  NaN   
       2017-10-31                                  0.0   

                    Net Cash from Acquisitions & Divestitures  \
Ticker Report Date                                              
A      2016-10-31                                 -26000000.0   
       2017-01-31                                 -69000000.0   
       2017-04-30                                         0.0   
       2017-07-31                                 -57000000.0   
       2017-10-31                                         0.0   

                    Net Cash from Investing Activities  Dividends Paid  \
Ticker Report Date                                                       
A      2016-10-31                          -78000000.0     -38000000.0   
       2017-01-31                         -101000000.0     -42000000.0   
       2017-04-30                          -43000000.0     -43000000.0   
       2017-07-31                         -101000000.0     -42000000.0   
       2017-10-31                          -60000000.0     -43000000.0   

                    Cash from (Repayment of) Debt  \
Ticker Report Date                                  
A      2016-10-31                      27000000.0   
       2017-01-31

In [5]:
print('Income Statement CSV data is: ', df_income.shape)
print('Balance Sheet CSV data is: ', df_balance.shape)
print('Cash Flow CSV data is: ', df_cashflow.shape)

Income Statement CSV data is:  (36685, 26)
Balance Sheet CSV data is:  (36685, 28)
Cash Flow CSV data is:  (36685, 26)


Number of rows of data is consistent. There are altogether 36685 rows to use.

In [6]:
# Merge the data together
# Define the column features where merge takes place
list_to_merge_on = ['Ticker', 'SimFinId', 'Currency', 'Fiscal Year', 'Report Date', 'Publish Date']

# Merge the income statement and balance sheet first
merge1 = pd.merge(df_income, df_balance, on = list_to_merge_on, how = 'inner')

# Merge previous result with cashflow statement
df_merged = pd.merge(merge1, df_cashflow, on = list_to_merge_on, how = 'inner')

# Reset the index
df_merged.reset_index(inplace=True)

# Make sure that the dates are in correct format
df_merged["Report Date"] = pd.to_datetime(df_merged["Report Date"])
df_merged["Publish Date"] = pd.to_datetime(df_merged["Publish Date"])

print('Merged data matrix shape is: ', df_merged.shape)

Merged data matrix shape is:  (36685, 74)


In [7]:
df_merged.head()

,Ticker,Report Date,SimFinId,Currency,Fiscal Year,Fiscal Period_x,Publish Date,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,...,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,2016-10-31,45846,USD,2016,Q4,2016-12-20,2018-12-20,324000000.0,328000000.0,...,234000000.0,-52000000.0,0.0,-26000000.0,-78000000.0,-38000000.0,27000000.0,-43000000.0,-56000000.0,90000000
1,A,2017-01-31,45846,USD,2017,Q1,2017-03-08,2018-03-06,322000000.0,326000000.0,...,116000000.0,-32000000.0,NaN,-69000000.0,-101000000.0,-42000000.0,89000000.0,-93000000.0,-58000000.0,-48000000
2,A,2017-04-30,45846,USD,2017,Q2,2017-06-06,2018-05-31,321000000.0,325000000.0,...,257000000.0,-43000000.0,NaN,0.0,-43000000.0,-43000000.0,52000000.0,-75000000.0,-67000000.0,148000000
3,A,2017-07-31,45846,USD,2017,Q3,2017-09-06,2018-08-30,321000000.0,326000000.0,...,228000000.0,-43000000.0,NaN,-57000000.0,-101000000.0,-42000000.0,39000000.0,32000000.0,29000000.0,174000000
4,A,2017-10-31,45846,USD,2017,Q4,2017-12-21,2018-12-20,324000000.0,327000000.0,...,288000000.0,-58000000.0,0.0,0.0,-60000000.0,-43000000.0,-70000000.0,8000000.0,-106000000.0,115000000


In [8]:
df_prices = sf.load_shareprices(variant='daily', market='us')

Dataset "us-shareprices-daily" on disk (26 days old).
- Loading from disk ... Done!


In [9]:
df_prices.reset_index(inplace=True)

In [10]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3157862 entries, 0 to 3157861
Data columns (total 11 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Ticker              object        
 1   Date                datetime64[ns]
 2   SimFinId            int64         
 3   Open                float64       
 4   Low                 float64       
 5   High                float64       
 6   Close               float64       
 7   Adj. Close          float64       
 8   Dividend            float64       
 9   Volume              int64         
 10  Shares Outstanding  float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(1)
memory usage: 265.0+ MB


In [11]:
df_prices.isnull().sum()

Ticker                      0
Date                        0
SimFinId                    0
Open                    21601
Low                     22052
High                    21166
Close                   21571
Adj. Close              21456
Dividend              3134252
Volume                      0
Shares Outstanding      44696
dtype: int64

## Function 1: to get price of a ticker (or find the nearest price given date)

In [12]:
# This is a function that takes in the ticker, start date and the prices dataframe
# It will return the ticker, price and date, and transaction value.

def getPrice (ticker, datestart, days_fwd, df=df_prices):
    
    # looks into a window of the next 5 days if date chosen falls on a non-trading day
    window = 5
    
    # returns all the rows between the start date and the window, filtered from df
    rows = df[
    (df["Date"].between(pd.to_datetime(datestart) + pd.Timedelta(days=days_fwd),\
                        pd.to_datetime(datestart) + pd.Timedelta(days=window + days_fwd)))\
        & (df["Ticker"]==ticker)]
    
    # if nothing between start date and window, return nothing
    # NaT stands for Not a Time
    if rows.empty:
        return [ticker, np.float("NaN"), np.datetime64('NaT'), np.float("NaN")]
    
    # else return the first row within the filtered df i.e. closest date
    # returns ticker, open price, date, turnover value (volume x open price)
    else:
        return [ticker, 
                rows.iloc[0]["Open"],\
                rows.iloc[0]["Date"],\
                rows.iloc[0]["Volume"]]

### To demonstrate how above function works

In [13]:
# get daily AAPL data in Sep 2016
df_prices.query("Ticker=='AAPL' & Date<'2016-10-01'")

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
10237,AAPL,2016-09-06,111052,26.98,26.88,27.07,26.93,25.11,NaN,107521564,2.155377e+10
10238,AAPL,2016-09-07,111052,26.96,26.77,27.19,27.09,25.26,NaN,169457312,2.155377e+10
10239,AAPL,2016-09-08,111052,26.81,26.31,26.82,26.38,24.60,NaN,212008104,2.155377e+10
10240,AAPL,2016-09-09,111052,26.16,25.78,26.43,25.78,24.04,NaN,186227936,2.155377e+10
10241,AAPL,2016-09-12,111052,25.66,25.63,26.43,26.36,24.58,NaN,181171080,2.155377e+10
10242,AAPL,2016-09-13,111052,26.88,26.81,27.20,26.99,25.16,NaN,248704760,2.155377e+10
10243,AAPL,2016-09-14,111052,27.18,27.15,28.26,27.94,26.06,NaN,449361272,2.155377e+10
10244,AAPL,2016-09-15,111052,28.46,28.37,28.93,28.89,26.94,NaN,362452708,2.155377e+10
10245,AAPL,2016-09-16,111052,28.78,28.51,29.03,28.73,26.79,NaN,319547644,2.155377e+10
10246,AAPL,2016-09-19,111052,28.80,28.31,29.05,28.39,26.48,NaN,188092184,2.155377e+10


In [14]:
# use function to get price data from AAPL 09 Sep 2016
getPrice('AAPL', '2016-09-09', 0, df_prices)

['AAPL', 26.16, Timestamp('2016-09-09 00:00:00'), 186227936]

In [15]:
# use function to get price date from AAPL 10 Sep 2016
# as 10 Sep 2016 is non trading day, function will find the price info from 12 Sep instead
getPrice('AAPL', '2016-09-10', 0, df_prices)

['AAPL', 25.66, Timestamp('2016-09-12 00:00:00'), 181171080]

In [16]:
# use function to get APPL price date 30 days in the future using the days_adj parameter
getPrice('AAPL', '2016-09-10', 30, df_prices)

['AAPL', 28.75, Timestamp('2016-10-10 00:00:00'), 144943824]

## Function 2
- Take in all fundamental data (fund_data) and all stock prices (price_data), with a days forward modifier
- Returns stock price info for report date and stock price one year (can change using modifier) from that date

In [17]:
def getReportDatePrices(fundmt_data, price_data, days_fwd=365):
    
    # We want to get a list of 2 lists
    # [(price at date) (price at date + days_fwd)]
    
    # creates a list of lists of len(fundmt_data) rows and 8 columns
    # will update with 4 
    # instantiate null dataframe first
    
    y = [[None]*8 for i in range(len(fundmt_data))]
    
    # We can use Publish Date or Report Date, but to get performance we should use Publish Date
    # We cannot take action on Report Date, but we can trade when info is being published
    
    date_column = 'Report Date' 

    # start from zero, loop through all rows and append to the empty null list
    
    i=0
    for index in range(len(fundmt_data)):
        y[i] = (getPrice(fundmt_data['Ticker'].iloc[index], fundmt_data[date_column].iloc[index], 0, price_data) +\
                getPrice(fundmt_data['Ticker'].iloc[index], fundmt_data[date_column].iloc[index], days_fwd, price_data))
        i=i+1
    return y

In [18]:
#getReportDatePrices(df_merged, df_prices, 365)

This will take quite a long time to run. Estimated time would be a few hours, and not efficient because it is looking at the 3 million rows of data from the df_prices dataset. Find another workaround which is to merge the fund_data and price_data to get a subset of the price data, before applying the getPrice function.

**At this point, we want to combine the fundamental data and the price data together, such that we have the price data on each Report Date. However, sometimes the Report Date falls on a weekend/non-trading day, so we need to backfill from a future date. Below we will attempt to do this.**

In [19]:
# all the fundemental data that we have combined so far
pd.set_option('display.max_columns', 200)
df_merged.head()

,Ticker,Report Date,SimFinId,Currency,Fiscal Year,Fiscal Period_x,Publish Date,Restated Date_x,Shares (Basic)_x,Shares (Diluted)_x,Revenue,Cost of Revenue,Gross Profit,Operating Expenses,"Selling, General & Administrative",Research & Development,Depreciation & Amortization_x,Operating Income (Loss),Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common),Fiscal Period_y,Restated Date_y,Shares (Basic)_y,Shares (Diluted)_y,"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,"Property, Plant & Equipment, Net",Long Term Investments & Receivables,Other Long Term Assets,Total Noncurrent Assets,Total Assets,Payables & Accruals,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity,Fiscal Period,Restated Date,Shares (Basic),Shares (Diluted),Net Income/Starting Line,Depreciation & Amortization_y,Non-Cash Items,Change in Working Capital,Change in Accounts Receivable,Change in Inventories,Change in Accounts Payable,Change in Other,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,2016-10-31,45846,USD,2016,Q4,2016-12-20,2018-12-20,324000000.0,328000000.0,1.111000e+09,-523000000.0,588000000.0,-405000000.0,-321000000.0,-84000000.0,NaN,183000000.0,-32000000.0,-16000000.0,151000000,NaN,151000000,-25000000.0,126000000,NaN,126000000,126000000,Q4,2017-12-21,324000000.0,328000000.0,2.289000e+09,631000000.0,533000000.0,3.635000e+09,639000000.0,135000000.0,3.385000e+09,4.159000e+09,7794000000,257000000.0,0.0,945000000,1.904000e+09,2.603000e+09,3548000000,9.165000e+09,-1.050800e+10,6.089000e+09,4.246000e+09,7794000000,Q4,2018-12-20,324000000.0,328000000.0,126000000.0,56000000.0,2000000.0,50000000.0,-52000000.0,4000000.0,12000000.0,86000000.0,234000000.0,-52000000.0,0.0,-26000000.0,-78000000.0,-38000000.0,27000000.0,-43000000.0,-56000000.0,90000000
1,A,2017-01-31,45846,USD,2017,Q1,2017-03-08,2018-03-06,322000000.0,326000000.0,1.067000e+09,-493000000.0,574000000.0,-368000000.0,-289000000.0,-79000000.0,NaN,206000000.0,-13000000.0,-16000000.0,193000000,NaN,193000000,-25000000.0,168000000,NaN,168000000,168000000,Q1,2017-03-08,322000000.0,326000000.0,2.241000e+09,653000000.0,551000000.0,3.635000e+09,653000000.0,133000000.0,3.451000e+09,4.237000e+09,7872000000,268000000.0,190000000.0,1089000000,1.802000e+09,2.483000e+09,3572000000,5.239000e+09,0.000000e+00,-4.530000e+08,4.300000e+09,7872000000,Q1,2018-03-06,322000000.0,326000000.0,168000000.0,55000000.0,45000000.0,-152000000.0,-31000000.0,-26000000.0,9000000.0,-104000000.0,116000000.0,-32000000.0,NaN,-69000000.0,-101000000.0,-42000000.0,89000000.0,-93000000.0,-58000000.0,-48000000
2,A,2017-04-30,45846,USD,2017,Q2,2017-06-06,2018-05-31,321000000.0,325000000.0,1.102000e+09,-510000000.0,592000000.0,-391000000.0,-307000000.0,-84000000.0,NaN,201000000.0,-10000000.0,-15000000.0,191000000,NaN,191000000,-27000000.0,164000000,NaN,164000000,164000000,Q2,2017-06-06,321000000.0,325000000.0,2.389000e+09,677000000.0,548000000.0,3.800000e+09,675000000.0,134000000.0,3.407000e+09,4.216000e+09,8016000000,265000000.0,241000000.0,1187000000,1.802000e+09,2.454000e+09,3641000000,5.242000e+09,0.000000e+00,-3.930000e+08,4.375000e+09,8016000000,Q2,2018-05-31,321000000.0,325000000.0,164000000.0,54000000.0,27000000.0,12000000.0,-17000000.0,-3000000.0,-3000000.0,35000000.0,257000000.0,-43000000.0,NaN,0.0,-43000000.0,-43000000.0,52000000.0,-750

In [20]:
# all the price data that we have combined so far
df_prices.head()

,Ticker,Date,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,A,2016-09-06,45846,46.90,46.72,47.11,46.95,44.72,NaN,1460928,324384755.0
1,A,2016-09-07,45846,47.07,46.86,47.17,47.11,44.87,NaN,1542576,324384755.0
2,A,2016-09-08,45846,47.07,46.94,47.16,47.02,44.79,NaN,884738,324384755.0
3,A,2016-09-09,45846,46.51,44.87,46.53,44.88,42.75,NaN,2507018,324384755.0
4,A,2016-09-12,45846,44.59,44.47,45.81,45.75,43.58,NaN,1835434,324384755.0


In [21]:
# We want to find the price of each ticker on each Report Date. To do this, we create a new dataframe using the above info.
df_report_date = df_merged[['Ticker', 'Report Date']]
df_price_date = df_prices[['Ticker', 'Date', 'Open', 'Volume']]

# merge the two subsets above using outer join
price_on_report_date = pd.merge(df_report_date, df_price_date, how='outer',
                                left_on=['Ticker','Report Date'], right_on=['Ticker','Date'])

In [22]:
price_on_report_date.head(10)

,Ticker,Report Date,Date,Open,Volume
0,A,2016-10-31,2016-10-31,43.33,1556762.0
1,A,2017-01-31,2017-01-31,48.04,2552612.0
2,A,2017-04-30,NaT,NaN,NaN
3,A,2017-07-31,2017-07-31,60.32,1038744.0
4,A,2017-10-31,2017-10-31,67.74,1609938.0
5,A,2018-01-31,2018-01-31,73.77,1949108.0
6,A,2018-04-30,2018-04-30,66.49,2277705.0
7,A,2018-07-31,2018-07-31,64.93,2539265.0
8,A,2018-10-31,2018-10-31,64.35,3820987.0
9,A,2019-01-31,2019-01-31,75.21,2951321.0


As we can see from above, some reporting dates do not have price data, so we need to backfill it with the closest price open. 

The steps to do this would be:
1. Fill the null values in "Date" with the "Report Date"
2. Sort the "Date" column
3. Backfill the price and volume data
4. Drop all the rows with nulls (price data that is not on Report Date)

In [23]:
# if null value found in "Date" column, fill with "Report Date" value, else continue
price_on_report_date['Date'] = np.where(price_on_report_date['Date'].isnull(), 
                                        price_on_report_date['Report Date'], 
                                        price_on_report_date['Date'])

In [24]:
# Sort the tickers by date first, then we do a backfill of price and volume data
price_on_report_date['Open'] = price_on_report_date.sort_values(['Ticker', 'Date'])['Open'].backfill()
price_on_report_date['Volume'] = price_on_report_date.sort_values(['Ticker', 'Date'])['Volume'].backfill()

In [25]:
# check that the price values have been backfilled correctly
price_on_report_date = price_on_report_date.sort_values(['Ticker', 'Date']).dropna()

In [26]:
# check for nulls
price_on_report_date.isnull().sum()

Ticker         0
Report Date    0
Date           0
Open           0
Volume         0
dtype: int64

In [27]:
# We can also drop Report Date since it is the same as the Date column
price_on_report_date = price_on_report_date.drop(columns='Report Date')

In [28]:
# check shape
price_on_report_date.shape

(36685, 4)

In [29]:
# merge the Price data to the Fundamental data to a df called result
result = pd.merge(df_merged, price_on_report_date, how='inner',
                   left_on=['Ticker', 'Report Date'],
                   right_on=['Ticker', 'Date'])

In [30]:
# check out the final resulting dataframe
result.isnull().sum()

Ticker                                  0
Report Date                             0
SimFinId                                0
Currency                                0
Fiscal Year                             0
                                     ... 
Net Cash from Financing Activities    247
Net Change in Cash                      0
Date                                    0
Open                                    0
Volume                                  0
Length: 77, dtype: int64

In [31]:
price_on_report_date.head()

,Ticker,Date,Open,Volume
0,A,2016-10-31,43.33,1556762.0
1,A,2017-01-31,48.04,2552612.0
2,A,2017-04-30,55.50,1897018.0
3,A,2017-07-31,60.32,1038744.0
4,A,2017-10-31,67.74,1609938.0


In [32]:
# This takes about 3-5 mins to run 35k rows
y = getReportDatePrices(df_merged, price_on_report_date)

C:\Users\finan\AppData\Local\Temp/ipykernel_28476/1699560653.py:18: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return [ticker, np.float("NaN"), np.datetime64('NaT'), np.float("NaN")]


In [33]:
column_names = ['Ticker1', 'Open1', 'Date1', 'Volume1',
                'Ticker2', 'Open2', 'Date2', 'Volume2']
y = pd.DataFrame(y, columns=column_names)

In [34]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36685 entries, 0 to 36684
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Ticker1  36685 non-null  object        
 1   Open1    36685 non-null  float64       
 2   Date1    36685 non-null  datetime64[ns]
 3   Volume1  36685 non-null  float64       
 4   Ticker2  36685 non-null  object        
 5   Open2    28051 non-null  float64       
 6   Date2    28051 non-null  datetime64[ns]
 7   Volume2  28051 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(2)
memory usage: 2.2+ MB


In [35]:
y.head()

,Ticker1,Open1,Date1,Volume1,Ticker2,Open2,Date2,Volume2
0,A,43.33,2016-10-31,1556762.0,A,67.74,2017-10-31,1609938.0
1,A,48.04,2017-01-31,2552612.0,A,73.77,2018-01-31,1949108.0
2,A,55.50,2017-04-30,1897018.0,A,66.49,2018-04-30,2277705.0
3,A,60.32,2017-07-31,1038744.0,A,64.93,2018-07-31,2539265.0
4,A,67.74,2017-10-31,1609938.0,A,64.35,2018-10-31,3820987.0


In [36]:
x = df_merged

In [37]:
# save the final file to csv
x.to_csv("./data/StocksFundamentals.csv")
y.to_csv("./data/StockPerformanceAnnual.csv")

In [40]:
pickle.dump(x, open('./data/fundamentals.pkl', 'wb'))
pickle.dump(y, open('./data/performance.pkl', 'wb'))